# Chapter 5 — Tool Use: Vertex AI Search

Use **Vertex AI Search** as a tool for an ADK agent.

**Prerequisites**: Requires a Google Cloud project with Vertex AI enabled and a Vertex AI Search data store.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

## Configure Vertex AI Search

Update the constants below with your Google Cloud project details.

In [ ]:
# TODO: Replace with your actual Google Cloud project details
GCP_PROJECT_ID = "your-project-id"
VERTEX_SEARCH_DATA_STORE_ID = "your-data-store-id"
GCP_LOCATION = "global"  # or your specific region

print(f"Project: {GCP_PROJECT_ID}")
print(f"Data Store: {VERTEX_SEARCH_DATA_STORE_ID}")
print(f"Location: {GCP_LOCATION}")

## Create Agent with Vertex AI Search Tool

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import VertexAiSearchTool

vertex_search_tool = VertexAiSearchTool(
    data_store_id=f"projects/{GCP_PROJECT_ID}/locations/{GCP_LOCATION}/collections/default_collection/dataStores/{VERTEX_SEARCH_DATA_STORE_ID}"
)

vertex_agent = Agent(
    name="vertex_search_agent",
    model="gemini-2.0-flash",
    description="An agent that searches through enterprise data using Vertex AI Search.",
    instruction="""You are an enterprise search assistant. Use the Vertex AI Search tool to find
relevant documents and information from the organization's data store. Provide accurate
answers with references to source documents.""",
    tools=[vertex_search_tool],
)

print("Vertex AI Search agent ready")

## Run the Agent

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=vertex_agent, app_name="vertex_demo", session_service=session_service)

session = await session_service.create_session(app_name="vertex_demo", user_id="user1")

# Adjust this query to match your data store content
request = types.Content(
    role="user",
    parts=[types.Part(text="Search for information about our company's data governance policies.")]
)

response = await runner.run_async(user_id="user1", session_id=session.id, new_message=request)

for event in response:
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text}")

## Key Takeaways

- **Vertex AI Search**: Provides RAG-like capabilities over your enterprise data
- Requires a Google Cloud project with Vertex AI and a configured data store
- Same ADK pattern: `Agent(tools=[vertex_search_tool]) → Runner → run_async()`